In [1]:
tag2id = {'PER': 1, 'ORG': 2, 'LOC': 3, 'MISC': 4, 'NCHUNK': 5, 'TIME': 6, 'PLACE': 7}
id2tag = {v:k for k, v in tag2id.items()}
id2tag

{1: 'PER', 2: 'ORG', 3: 'LOC', 4: 'MISC', 5: 'NCHUNK', 6: 'TIME', 7: 'PLACE'}

In [2]:
label2id = {
    'O': 0, 
    **{f'B-{k}': 2*v - 1 for k, v in tag2id.items()},
    **{f'I-{k}': 2*v for k, v in tag2id.items()}
}

id2label = {v:k for k, v in label2id.items()}

id2label

{0: 'O',
 1: 'B-PER',
 3: 'B-ORG',
 5: 'B-LOC',
 7: 'B-MISC',
 9: 'B-NCHUNK',
 11: 'B-TIME',
 13: 'B-PLACE',
 2: 'I-PER',
 4: 'I-ORG',
 6: 'I-LOC',
 8: 'I-MISC',
 10: 'I-NCHUNK',
 12: 'I-TIME',
 14: 'I-PLACE'}

In [3]:
!pip install datasets
!pip install transformers==4.28.0
!pip install sacremoses
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from datasets import Dataset
train_ds = Dataset.from_json("/content/drive/MyDrive/annotations.train.jsonlines")
val_ds = Dataset.from_json("/content/drive/MyDrive/annoatations.validation.jsonlines")

train_ds[0]

{'tags': [{'end': 32, 'start': 19, 'tag': 'PER'},
  {'end': 32, 'start': 6, 'tag': 'NCHUNK'},
  {'end': 152, 'start': 143, 'tag': 'NCHUNK'},
  {'end': 225, 'start': 211, 'tag': 'NCHUNK'},
  {'end': 79, 'start': 45, 'tag': 'NCHUNK'}],
 'id': 'train_253',
 'text': "Selon l'ethnologue Maurice Duval, « dire que ce mouvement de la gauche radicale est « une secte », ce n'est pas argumenter légitimement contre ses idées, mais c'est suggérer qu'il est malfaisant, malsain et que sa disparition serait souhaitable »."}

In [5]:
for i in range(3):
    example = train_ds[i]
    print(f"\n{example['text']}")
    for tag_item in example["tags"]:
        print(tag_item["tag"].ljust(10), "-", example["text"][tag_item["start"]: tag_item["end"]])



Selon l'ethnologue Maurice Duval, « dire que ce mouvement de la gauche radicale est « une secte », ce n'est pas argumenter légitimement contre ses idées, mais c'est suggérer qu'il est malfaisant, malsain et que sa disparition serait souhaitable ».
PER        - Maurice Duval
NCHUNK     - l'ethnologue Maurice Duval
NCHUNK     - ses idées
NCHUNK     - sa disparition
NCHUNK     - ce mouvement de la gauche radicale

Adolescent, il joue de la basse dans un groupe de surf music, commence à composer et s'intéresse aux œuvres de musique contemporaine de compositeurs comme Charles Ives, Karlheinz Stockhausen, Mauricio Kagel, ou encore John Cage.
PER        - Charles Ives
PER        - Karlheinz Stockhausen
PER        - Mauricio Kagel
PER        - John Cage
NCHUNK     - un groupe de surf music
NCHUNK     - œuvres de musique contemporaine de compositeurs comme Charles Ives, Karlheinz Stockhausen, Mauricio Kagel, ou encore John Cage

Metacritic ", qui détermine une moyenne pondérée entre 0 et 100 b

In [6]:
from transformers import AutoTokenizer
camembert = AutoTokenizer.from_pretrained("camembert-base")
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
tokens = tokenizer("On y va.", return_offsets_mapping=True)
tokens.offset_mapping

[(0, 0), (0, 2), (3, 4), (5, 7), (7, 8), (0, 0)]

In [7]:
def get_token_role_in_span(token_start: int, token_end: int, span_start: int, span_end: int):
    """
    Check if the token is inside a span.
    Args:
      - token_start, token_end: Start and end offset of the token
      - span_start, span_end: Start and end of the span
    Returns:
      - "B" if beginning
      - "I" if inner
      - "O" if outer
      - "N" if not valid token (like <SEP>, <CLS>, <UNK>)
    """
    if token_end <= token_start:
        return "N"
    if token_start < span_start or token_end > span_end:
        return "O"
    if token_start > span_start:
        return "I"
    else:
        return "B"

MAX_LENGTH = 256

def tokenize_and_adjust_labels(sample):
    """
    Args:
        - sample (dict): {"id": "...", "text": "...", "tags": [{"start": ..., "end": ..., "tag": ...}, ...]
    Returns:
        - The tokenized version of `sample` and the labels of each token.
    """
    # Tokenize the text, keep the start and end positions of tokens with `return_offsets_mapping` option
    # Use max_length and truncation to ajust the text length
    tokenized = tokenizer(sample["text"], 
                          return_offsets_mapping=True, 
                          padding="max_length", 
                          max_length=MAX_LENGTH,
                          truncation=True)
    
    # We are doing a multilabel classification task at each token, we create a list of size len(label2id)=13 
    # for the 13 labels
    labels = [[0 for _ in label2id.keys()] for _ in range(MAX_LENGTH)]
    
    # Scan all the tokens and spans, assign 1 to the corresponding label if the token lies at the beginning
    # or inside the spans
    for (token_start, token_end), token_labels in zip(tokenized["offset_mapping"], labels):
        for span in sample["tags"]:
            role = get_token_role_in_span(token_start, token_end, span["start"], span["end"])
            if role == "B":
                token_labels[label2id[f"B-{span['tag']}"]] = 1
            elif role == "I":
                token_labels[label2id[f"I-{span['tag']}"]] = 1
    
    return {**tokenized, "labels": labels}

In [8]:
tokenized_train_ds = train_ds.map(tokenize_and_adjust_labels, remove_columns=train_ds.column_names)
tokenized_val_ds = val_ds.map(tokenize_and_adjust_labels, remove_columns=val_ds.column_names)

Map:   0%|          | 0/329 [00:00<?, ? examples/s]

Map:   0%|          | 0/83 [00:00<?, ? examples/s]

In [9]:
sample = tokenized_train_ds[0]
sample["labels"]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [10]:
sample = tokenized_train_ds[0]
print("--------Token---------|--------Labels----------")
for token_id, token_labels in zip(sample["input_ids"], sample["labels"]):
    # Decode the token_id into text
    token_text = tokenizer.decode(token_id)
    
    # Retrieve all the indices corresponding to the "1" at each token, decode them to label name
    labels = [id2label[label_index] for label_index, value in enumerate(token_labels) if value==1]
    
    # Decode those indices into label name
    print(f" {token_text:20} | {labels}")
    
    # Finish when we meet the end of sentence.
    if token_text == "</s>": 
        break

--------Token---------|--------Labels----------
 <s>                  | []
 Selon                | []
 l                    | ['B-NCHUNK']
 '                    | ['I-NCHUNK']
 ethno                | ['I-NCHUNK']
 logue                | ['I-NCHUNK']
 Maurice              | ['B-PER', 'I-NCHUNK']
 Duval                | ['I-PER', 'I-NCHUNK']
 ,                    | []
 «                    | []
 dire                 | []
 que                  | []
 ce                   | ['B-NCHUNK']
 mouvement            | ['I-NCHUNK']
 de                   | ['I-NCHUNK']
 la                   | ['I-NCHUNK']
 gauche               | ['I-NCHUNK']
 radicale             | ['I-NCHUNK']
 est                  | []
 «                    | []
 une                  | []
 secte                | []
 »,                   | []
 ce                   | []
 n                    | []
 '                    | []
 est                  | []
 pas                  | []
 argument             | []
 er                   | []
 lég

In [11]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, padding=True)

In [12]:
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix

n_labels = len(id2label)

def divide(a: int, b: int):
    return a / b if b > 0 else 0

def compute_metrics(p):
    """
    Customize the `compute_metrics` of `transformers`
    Args:
        - p (tuple):      2 numpy arrays: predictions and true_labels
    Returns:
        - metrics (dict): f1 score on 
    """
    # (1)
    predictions, true_labels = p
    
    # (2)
    predicted_labels = np.where(predictions > 0, np.ones(predictions.shape), np.zeros(predictions.shape))
    metrics = {}
    
    # (3)
    cm = multilabel_confusion_matrix(true_labels.reshape(-1, n_labels), predicted_labels.reshape(-1, n_labels))
    
    # (4) 
    for label_idx, matrix in enumerate(cm):
        if label_idx == 0:
            continue # We don't care about the label "O"
        tp, fp, fn = matrix[1, 1], matrix[0, 1], matrix[1, 0]
        precision = divide(tp, tp + fp)
        recall = divide(tp, tp + fn)
        f1 = divide(2 * precision * recall, precision + recall)
        metrics[f"f1_{id2label[label_idx]}"] = f1
        
    # (5)
    macro_f1 = sum(list(metrics.values())) / (n_labels - 1)
    metrics["macro_f1"] = macro_f1
        
    return metrics

In [13]:
import transformers
model = transformers.AutoModelForTokenClassification.from_pretrained("camembert-base")
type(model)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForTokenClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream tas

transformers.models.camembert.modeling_camembert.CamembertForTokenClassification

In [14]:
transformers.models.camembert.modeling_camembert.CamembertModel.__mro__ # This lists the parent classes that CamembertModel inhe

(transformers.models.camembert.modeling_camembert.CamembertModel,
 transformers.models.camembert.modeling_camembert.CamembertPreTrainedModel,
 transformers.modeling_utils.PreTrainedModel,
 torch.nn.modules.module.Module,
 transformers.modeling_utils.ModuleUtilsMixin,
 transformers.generation.utils.GenerationMixin,
 transformers.utils.hub.PushToHubMixin,
 object)

In [15]:
from torch import nn
loss_fct = nn.BCEWithLogitsLoss()
#loss = loss_fct(logits, labels.float())

In [16]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import RobertaPreTrainedModel, RobertaModel
from transformers.utils import (
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    logging,
    replace_return_docstrings,
)
from transformers.models.roberta.modeling_roberta import (
    ROBERTA_INPUTS_DOCSTRING,
    ROBERTA_START_DOCSTRING,
    RobertaEmbeddings
)
from typing import Optional, Union, Tuple
from transformers.modeling_outputs import TokenClassifierOutput
import torch
from torch import nn

class RobertaForSpanCategorization(RobertaPreTrainedModel):
    _keys_to_ignore_on_load_unexpected = [r"pooler"]
    _keys_to_ignore_on_load_missing = [r"position_ids"]
    
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # Initialize weights and apply final processing
        self.post_init()
    
    @add_start_docstrings_to_model_forward(ROBERTA_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], TokenClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the token classification loss. Indices should be in `[0, ..., config.num_labels - 1]`.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels.float())
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output
        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [17]:
training_args = TrainingArguments(
    output_dir="./models/fine_tune_bert_output_span_cat",
    evaluation_strategy="epoch",
    learning_rate=2.5e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps = 100,
    save_strategy='epoch',
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    log_level='critical',
    seed=12345
)

In [18]:
def model_init():
    # For reproducibility
    return RobertaForSpanCategorization.from_pretrained("camembert-base", id2label=id2label, label2id=label2id)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 B-per,F1 I-per,F1 B-org,F1 I-org,F1 B-loc,F1 I-loc,F1 B-misc,F1 I-misc,F1 B-nchunk,F1 I-nchunk,F1 B-time,F1 I-time,F1 B-place,F1 I-place,Macro F1
1,No log,0.331786,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
2,No log,0.217084,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
3,No log,0.148405,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
4,No log,0.110007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
5,0.233000,0.087497,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
6,0.233000,0.073911,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
7,0.233000,0.065578,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
8,0.233000,0.060613,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
9,0.233000,0.057878,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
10,0.068500,0.056999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000


TrainOutput(global_step=210, training_loss=0.1463300531818753, metrics={'train_runtime': 260.576, 'train_samples_per_second': 12.626, 'train_steps_per_second': 0.806, 'total_flos': 429883684070400.0, 'train_loss': 0.1463300531818753, 'epoch': 10.0})

In [19]:
trainer.model.save_pretrained("./models/fine_tune_bert_output_span_cat")


In [20]:
model = RobertaForSpanCategorization.from_pretrained("./models/fine_tune_bert_output_span_cat")
tokenizer = AutoTokenizer.from_pretrained("camembert-base")

In [21]:
def get_offsets_and_predicted_tags(example: str, model, tokenizer, threshold=0):
    """
    Get prediction of model on example, using tokenizer
    Args:
      - example (str): The input text
      - model: The span categorizer
      - tokenizer: The tokenizer
      - threshold: The threshold to decide whether the token should belong to the label. Default to 0, which corresponds to probability 0.5.
    Returns:
      - List of (token, tags, offset) for each token.
    """
    # Tokenize the sentence to retrieve the tokens and offset mappings
    raw_encoded_example = tokenizer(example, return_offsets_mapping=True)
    encoded_example = tokenizer(example, return_tensors="pt")
    
    # Call the model. The output LxK-tensor where L is the number of tokens, K is the number of classes
    out = model(**encoded_example)["logits"][0]
    
    # We assign to each token the classes whose logit is positive
    predicted_tags = [[i for i, l in enumerate(logit) if l > threshold] for logit in out]
    
    return [{"token": token, "tags": tag, "offset": offset} for (token, tag, offset) 
            in zip(tokenizer.batch_decode(raw_encoded_example["input_ids"]), 
                   predicted_tags, 
                   raw_encoded_example["offset_mapping"])]

In [22]:
example = "Du coup, la menace des feux de forêt est permanente, après les incendies dévastateurs de juillet dans le sud-ouest de la France, en Espagne, au Portugal ou en Grèce. Un important feu de forêt a éclaté le 24 juillet dans le parc national de la Suisse de Bohême, à la frontière entre la République tchèque et l'Allemagne, où des records de chaleur ont été battus (36,4C). Un millier d'hectares ont déjà été touchés. Lundi, les pompiers espéraient que l'incendie pourrait être maîtrisé en quelques jours."
for item in get_offsets_and_predicted_tags(example, model, tokenizer):
    print(f"""{item["token"]:15} - {item["tags"]}""")

<s>             - []
Du              - []
coup            - []
,               - []
la              - []
menace          - []
des             - []
feux            - []
de              - []
forêt           - []
est             - []
permanente      - []
,               - []
après           - []
les             - []
incendie        - []
s               - []
dévastateur     - []
s               - []
de              - []
juillet         - []
dans            - []
le              - []
sud             - []
-               - []
ouest           - []
de              - []
la              - []
France          - []
,               - []
en              - []
Espagne         - []
,               - []
au              - []
Portugal        - []
ou              - []
en              - []
Grèce           - []
.               - []
Un              - []
important       - []
feu             - []
de              - []
forêt           - []
a               - []
éclaté          - []
le              - []
24           

In [23]:
def get_tagged_groups(example: str, model, tokenizer):
    """
    Get prediction of model on example, using tokenizer
    Returns:
    - List of spans under offset format {"start": ..., "end": ..., "tag": ...}, sorted by start, end then tag.
    """
    offsets_and_tags = get_offsets_and_predicted_tags(example, model, tokenizer)
    predicted_offsets = {l: [] for l in tag2id}
    last_token_tags = []
    for item in offsets_and_tags:
        (start, end), tags = item["offset"], item["tags"]
        
        for label_id in tags:
            label = id2label[label_id]
            tag = label[2:] # "I-PER" => "PER"
            if label.startswith("B-"):
                predicted_offsets[tag].append({"start": start, "end": end})
            elif label.startswith("I-"):
                # If "B-" and "I-" both appear in the same tag, ignore as we already processed it
                if label2id[f"B-{tag}"] in tags:
                    continue
                
                if label_id not in last_token_tags and label2id[f"B-{tag}"] not in last_token_tags:
                    predicted_offsets[tag].append({"start": start, "end": end})
                else:
                    predicted_offsets[tag][-1]["end"] = end
        
        last_token_tags = tags
        
    flatten_predicted_offsets = [{**v, "tag": k, "text": example[v["start"]:v["end"]]} 
                                 for k, v_list in predicted_offsets.items() for v in v_list if v["end"] - v["start"] >= 3]
    flatten_predicted_offsets = sorted(flatten_predicted_offsets, 
                                       key = lambda row: (row["start"], row["end"], row["tag"]))
    return flatten_predicted_offsets

print(example)
get_tagged_groups(example, model, tokenizer)

Du coup, la menace des feux de forêt est permanente, après les incendies dévastateurs de juillet dans le sud-ouest de la France, en Espagne, au Portugal ou en Grèce. Un important feu de forêt a éclaté le 24 juillet dans le parc national de la Suisse de Bohême, à la frontière entre la République tchèque et l'Allemagne, où des records de chaleur ont été battus (36,4C). Un millier d'hectares ont déjà été touchés. Lundi, les pompiers espéraient que l'incendie pourrait être maîtrisé en quelques jours.


[]